* Author: HZQ
* Last modified: 2018/2/24


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p driver
!google-drive-ocamlfuse driver
import os

os.chdir("driver/Colab Notebooks/CategoryRecom_kaggle/")
!ls

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
data	  driver   model      requirements.txt	timeline
download  LICENSE  ReadMe.md  submit		utils


In [0]:
!ls

driver	sample_data


# Main Parts

In [0]:
# Used for better debug experience
%xmode Plain
# %pdb on

Exception reporting mode: Plain


In [0]:
import datetime
import gc, sys
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings

from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0']

In [0]:
DATA_ROOT_PATH = "./data/"

data_train_path = DATA_ROOT_PATH + "data_raw/train/train.csv"
# data_train_path = DATA_ROOT_PATH + "data_raw/train/train_20190218.csv"
data_test_path = DATA_ROOT_PATH + "data_raw/test/test.csv"
data_valid_path = DATA_ROOT_PATH + "data_raw/valid/valid.csv"
data_info_path = DATA_ROOT_PATH  + "data_raw/Data_Dictionary.xlsx"
data_merchants_path = DATA_ROOT_PATH + "data_raw/merchants.csv"
# data_merchants_path = DATA_ROOT_PATH + "data_raw/merchants_20190219.csv"
data_his_trans_path = DATA_ROOT_PATH + "data_raw/historical_transactions_20190218.csv"
data_new_trans_path = DATA_ROOT_PATH + "data_raw/new_merchant_transactions.csv"

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [0]:
# Re-construct 2019/2/21: create transaction couts, state/city counts for each card id in two transaction files 
def groupby_row2(each_row1):
#     each_row1_part = each_row1.loc[:, ["city_id", "merchant_category_id", "merchant_id"]]
    each_row1_part = each_row1.loc[:, ["state_id", "city_id", "merchant_category_id", "merchant_id"]]
    del each_row1
    gc.collect()
    sys._clear_type_cache()
    trans_count = each_row1_part.shape[0]
    state_count = each_row1_part["state_id"].value_counts().shape[0]
    city_count = each_row1_part["city_id"].value_counts().shape[0]
    merchant_category_count = each_row1_part["merchant_category_id"].value_counts().shape[0]
    merchant_count = each_row1_part["merchant_id"].value_counts().shape[0]
#     print(each_row1_part)
#     print("Checking transacation count: {}".format(trans_count))
    
    gc.collect()
    sys._clear_type_cache()    
        
    return trans_count, state_count, city_count, merchant_category_count, merchant_count      


def count_by_card_id(df):
    feature_new_df = df.groupby("card_id").apply(groupby_row2)
    del df
    
    return feature_new_df

In [0]:
hist_reader = pd.read_csv(data_his_trans_path, chunksize=6000000)   # Note: modify the chunksize if crash
feats_new_list = []
indent = 1
for each_chunk in hist_reader:
    print("[Output] expected to be end in 5 runs, this is {} run...".format(indent))
    each_chunk = reduce_mem_usage(each_chunk)      
    feats_new_series = count_by_card_id(each_chunk)
    print("Checking each output: {}".format(feats_new_series))
    feats_new_list.append(feats_new_series)
    indent += 1
    gc.collect()
sys._clear_type_cache()    

del each_chunk
del hist_reader
feats_new_series = pd.concat(feats_new_list, ignore_index=True)

gc.collect()
sys._clear_type_cache()

feats_new_series.to_csv(DATA_ROOT_PATH + "data_raw/feats_new_20190224csv", index=False)
# feats_new_series = pd.read_csv(DATA_ROOT_PATH + "data_raw/feats_new_20190224csv", header=0)

In [0]:
print("Total number of part is {}".format(len(feats_new_list)))
index_total = feats_new_list[0].index.append(feats_new_list[1].index).append(feats_new_list[2].index)
index_series = pd.Series(index_total)
index_series.to_csv(DATA_ROOT_PATH + "data_raw/his_trans_index_after_agg.csv", index=False)
feats_new_df = pd.DataFrame({"card_id": index_series.values})

ind = 0
feats_new_list, feats_new_list2 = \
    [each_row for each_row in feats_new_series.values], []

for each_feat in zip(*feats_new_list):
#     print(each_feat)
    # if ind == 2:
        # break
    if not("(" in each_feat) and not(")" in each_feat) and not("," in each_feat):    
        feats_new_list2.append(each_feat)
    ind += 1
    gc.collect()
    sys._clear_type_cache()
assert len(feats_new_list2) == 5

ind = 0
feats_new = \
    ["his_trans_count", 
     "his_state_id_count", 
     "his_city_id_count", 
     "his_merchant_category_id_count", 
     "his_merchant_id_count"]
for each_feat_new in feats_new[:]:
    feats_new_df[each_feat_new] = feats_new_list2[ind]
    print("Finish {} feat dealing process...".format(ind+1))
    ind += 1
    gc.collect()
    sys._clear_type_cache()

feats_new_df["his_trans_count"] = \
    feats_new_df["his_trans_count"].astype("int")
feats_new_df["his_state_id_count"] = \
    feats_new_df["his_state_id_count"].astype("int")
feats_new_df["his_city_id_count"] = \
    feats_new_df["his_city_id_count"].astype("int")
feats_new_df["his_merchant_category_id_count"] = \
    feats_new_df["his_merchant_category_id_count"].astype("int")
feats_new_df["his_merchant_id_count"] = \
    feats_new_df["his_merchant_id_count"].astype("int")
feats_new_df.to_csv(DATA_ROOT_PATH + "data_raw/feats_new_20190223_2.csv", index=False)